In [1]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import FastText
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, BatchNormalization, Dropout, Input, Activation
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import np_utils
from numpy.random import seed
import math, nltk, random, keras
from sklearn import preprocessing

2023-03-17 09:14:59.828201: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 09:15:00.021751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-17 09:15:00.021768: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-17 09:15:01.770312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
nltk.download('conll2000')

[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/chitrang/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


True

In [3]:
from nltk.corpus import conll2000

In [4]:
dataset = list(conll2000.tagged_sents(tagset='universal'))

In [5]:
print("Tagged sentences in dataset: ", len(dataset))
print("Tagged words in dataset:", len([item for sublist in dataset for item in sublist]))

Tagged sentences in dataset:  10948
Tagged words in dataset: 259104


In [6]:
def tag_sequence(sentences):
    return [[t for w, t in sentence] for sentence in sentences]

def text_sequence(sentences):
    return [[w for w, t in sentence] for sentence in sentences]
def untag(tagged_sentence):
    return [w for w, _ in tagged_sentence]

def untag_pos(tagged_sentence):
    return [t for _, t in tagged_sentence]

In [7]:
embs_path = '/home/chitrang/Downloads/wiki-news-300d-1M.vec'
embeddings = KeyedVectors.load_word2vec_format(embs_path, binary=False)
dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)
np.random.seed(3)
oov = np.random.uniform(-0.25, 0.25, dim)

In [8]:
def features_embs(sentence, index, window=2):
    unknown=0
    vec = np.array([])
    for i in range(index-window, index+window+1):
#         if i < 0:
#             vec = np.append(vec, pad)
#         if i > len(sentence)-1:
#             vec = np.append(vec, pad)
        try:
            vec = np.append(vec, embeddings[sentence[i]])
        except:
            vec = np.append(vec, oov)
            unknown += 1
    return vec, unknown

In [9]:
def transform_to_dataset(tagged_sentences, window):
    i=0
    X, y = [], []
    for doc_index, tagged in enumerate(tagged_sentences):
        for index in range(len(tagged)):
            X.append(features_embs(untag(tagged), index, window)[0])
            y.append(tagged[index][1])
            k = features_embs(untag(tagged), index, window)[1]
            i += k
    return X, y, i

def transform_test_sentence(sentence, window):
    X = []
    for index in range(len(sentence)):
            X.append(features_embs(sentence, index, window))
    return X

def vectorize(train, window=2):
    X_train, y_train, unk_tr = transform_to_dataset(train, window=window)
    X_train = [x for x in X_train]
    X_train = np.asarray(X_train)
    return X_train, y_train

In [10]:
X_train, y_train = vectorize(dataset)

In [11]:
# Dataset split: Train - Validation - Test
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [12]:
print(X_train.shape, X_val.shape, X_test.shape)

(209873, 1500) (23320, 1500) (25911, 1500)


In [13]:
classes = sorted(list(set(y_train)))
print(classes)

['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


In [14]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)
y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)
y_test = le.transform(y_test)
y_test = keras.utils.to_categorical(y_test)
print(y_train.shape)

(209873, 12)


In [15]:
model = Sequential()

model.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               768512    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 12)                780       
                                                                 
Total params: 802,124
Trainable params: 802,124
Non-trainable params: 0
_________________________________________________________________


2023-03-17 09:19:38.329325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 09:19:38.329514: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-17 09:19:38.329571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-17 09:19:38.329837: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-17 09:19:38.329879: W tensorflow/c

In [16]:
nb_epoch = 7
batch_size = 128
early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 5)

In [17]:
history = model.fit(X_train, y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_val, y_val),
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/7


2023-03-17 09:22:23.416992: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1259238000 exceeds 10% of free system memory.


1640/1640 [==============================] - 17s 10ms/step - loss: 0.2660 - accuracy: 0.9209 - val_loss: 0.0834 - val_accuracy: 0.9756
Epoch 2/7
1640/1640 [==============================] - 16s 10ms/step - loss: 0.1058 - accuracy: 0.9686 - val_loss: 0.0681 - val_accuracy: 0.9797
Epoch 3/7
1640/1640 [==============================] - 19s 12ms/step - loss: 0.0819 - accuracy: 0.9750 - val_loss: 0.0659 - val_accuracy: 0.9791
Epoch 4/7
1640/1640 [==============================] - 20s 12ms/step - loss: 0.0678 - accuracy: 0.9791 - val_loss: 0.0597 - val_accuracy: 0.9824
Epoch 5/7
1640/1640 [==============================] - 19s 11ms/step - loss: 0.0589 - accuracy: 0.9817 - val_loss: 0.0615 - val_accuracy: 0.9818
Epoch 6/7
1640/1640 [==============================] - 19s 11ms/step - loss: 0.0512 - accuracy: 0.9837 - val_loss: 0.0601 - val_accuracy: 0.9820
Epoch 7/7
1640/1640 [==============================] - 18s 11ms/step - loss: 0.0460 - accuracy: 0.9852 - val_loss: 0.0647 - val_accuracy: 0.

In [23]:
model.save("mlp_model.h5", save_format='h5')

In [24]:
model = keras.models.load_model("mlp_model.h5")

In [18]:
best_epoch = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
print('Best epoch index:', best_epoch)

Best epoch index: 6


In [19]:
scores = model.evaluate(X_test, y_test, verbose=1)

810/810 [==============================] - 2s 2ms/step - loss: 0.0662 - accuracy: 0.9824


In [20]:
y_pred = model.predict(X_test, batch_size=1, verbose=1)

25911/25911 [==============================] - 24s 907us/step


In [21]:
print("Accuracy: {0:.4f}".format(scores[1]))

Accuracy: 0.9824


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()